In [0]:
# 1) Funkcija za izračun komplementarnega drugega Zagrebškega indeksa
def komplementarni_drugi_zagreb(G):
    """
    Izračuna CM2(G) = sum_{uv in E(G)} |stopnja(u)^2 - stopnja(v)^2|.
    """
    return sum(abs(G.degree(u)**2 - G.degree(v)**2) for u, v in G.edges(labels=False))

# 2) Funkcija za generacijo grafa oblike K_k + nepovezani_(n-k)
def povezava_polni_z_nepovezanimi(n, k):
    Polni = graphs.CompleteGraph(k)
    Nepovezani = Graph(n - k)  # nepovezan graf z n-k izoliranimi vozlišči
    Polni.relabel({i: i for i in range(k)}, inplace=True)
    Nepovezani.relabel({i: i + k for i in range(n - k)}, inplace=True)
    G = Graph()
    G.add_vertices(range(n))
    G.add_edges(Polni.edges())
    # Nepovezani nima povezav
    # Dodaj vse robove med Polni in Nepovezani (join)
    for u in range(k):
        for v in range(k, n):
            G.add_edge(u, v)
    return G

# 3) Ali je graf izomorfen grafu povezava_polni_z_nepovezanimi(n, k)
def je_povezava_polni_z_nepovezanimi(G):
    n = G.order()
    for k in range(1, n):
        H = povezava_polni_z_nepovezanimi(n, k)
        if G.is_isomorphic(H):
            return True, k
    return False, None

# 4) Empirični test za vse povezane grafe danega reda n
def testiraj_grafe(n):
    rezultati = []
    for G in graphs.nauty_geng(f"-c {n}"):
        G = Graph(G)
        vrednost = komplementarni_drugi_zagreb(G)
        je_join, k_val = je_povezava_polni_z_nepovezanimi(G)
        rezultati.append({
            "graf": G,
            "vrednost": vrednost,
            "je_join": je_join,
            "k": k_val
        })
    max_vrednost = max(r["vrednost"] for r in rezultati)
    min_vrednost = min(r["vrednost"] for r in rezultati)
    max_grafi = [r for r in rezultati if r["vrednost"] == max_vrednost]
    min_grafi = [r for r in rezultati if r["vrednost"] == min_vrednost]
    return rezultati, max_grafi, min_grafi

# 5) Izpis rezultatov in tabela v slovenščini, ter risanje grafa
def generiraj_latex_tabelo(seznam_n):
    latex_izpis = ""
    for n in seznam_n:
        rezultati, max_grafi, min_grafi = testiraj_grafe(n)
        latex_izpis += f"\\section*{{Rezultati za $n = {n}$}}\n"
        latex_izpis += "\\begin{tabular}{|c|c|c|c|}\n"
        latex_izpis += "\\hline\n"
        latex_izpis += "Tip & $cM_2$ vrednost & Je povezava? & $k$ \\\\\n"
        latex_izpis += "\\hline\n"
        for r in min_grafi:
            join_str = "Da" if r["je_join"] else "Ne"
            k_val = r['k'] if r['k'] is not None else "-"
            latex_izpis += f"Minimalni & {r['vrednost']} & {join_str} & {k_val} \\\\\n"
        for r in max_grafi:
            join_str = "Da" if r["je_join"] else "Ne"
            k_val = r['k'] if r['k'] is not None else "-"
            latex_izpis += f"Maksimalni & {r['vrednost']} & {join_str} & {k_val} \\\\\n"
        latex_izpis += "\\hline\n"
        latex_izpis += "\\end{tabular}\n\n"
    return latex_izpis

# Primer uporabe za n=5,6,7,8
seznam_n = [5, 6, 7, 8]
latex_tabela = generiraj_latex_tabelo(seznam_n)
print(latex_tabela)

# Funkcija za risanje grafa
def narisi_povezavni_graf(n, k):
    G = povezava_polni_z_nepovezanimi(n, k)
    G.show(vertex_labels=True, figsize=5)

# Nariši graf za n=5, k=2
narisi_povezavni_graf(5, 2)
